In [3]:
%pip install pandas
%pip install scikit-learn
%pip install imblearn
%pip install numpy
%pip install matplotlib
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sentence_transformers import SentenceTransformer
import numpy as np


# Đọc dữ liệu
file_path = "./dataset_vi/Amazon_Product_Review_full_vi_cleaned.csv"
data = pd.read_csv(file_path)

data.columns = data.columns.str.strip()

# Kết hợp headline và body
data['full_review'] = data['review_headline'].fillna('') + ' ' + data['review_body'].fillna('')

# Tiền xử lý
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z0-9À-ỹ\s]', '', text)
    return text

data['full_review'] = data['full_review'].apply(preprocess_text)
data = data.dropna(subset=['full_review', 'sentiment'])
data['sentiment'] = data['sentiment'].astype(int)

X = data['full_review']
y = data['sentiment']

# Tách train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Dùng embedding mạnh hơn
embedder = SentenceTransformer('all-mpnet-base-v2')  # Hoặc thử 'paraphrase-multilingual-mpnet-base-v2' nếu muốn đa ngôn ngữ
X_train_emb = embedder.encode(X_train.tolist(), show_progress_bar=True)
X_test_emb = embedder.encode(X_test.tolist(), show_progress_bar=True)

# GridSearchCV cho SVM
param_grid = {
    'C': [0.1, 1],
    'gamma': ['scale', 0.01, 0.1, 1],
    'kernel': ['rbf']
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
svm = SVC(class_weight='balanced', random_state=42)
grid = GridSearchCV(svm, param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=2)
grid.fit(X_train_emb, y_train)

print("Best parameters:", grid.best_params_)
print("Best CV accuracy:", grid.best_score_)

# Đánh giá trên tập test
y_pred = grid.predict(X_test_emb)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))




Batches: 100%|██████████| 64/64 [00:05<00:00, 12.01it/s]


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Best CV accuracy: 0.8651989484410464
Test Accuracy: 0.8585558852621167
Classification report:
               precision    recall  f1-score   support

           0       0.84      0.89      0.86      1011
           1       0.88      0.83      0.85      1011

    accuracy                           0.86      2022
   macro avg       0.86      0.86      0.86      2022
weighted avg       0.86      0.86      0.86      2022

Confusion matrix:
 [[898 113]
 [173 838]]


In [5]:

# 9. Kiểm thử với các câu thực tế
test_sentences = [
    # Nhóm 1: Dự đoán lớp 0 (tiêu cực)
    "Tôi cực kỳ thất vọng với sản phẩm này. Nó ngừng hoạt động chỉ sau hai ngày và dịch vụ khách hàng không hỗ trợ gì khi tôi cố gắng yêu cầu đổi trả.",
    "Đây chắc chắn là lần mua hàng tệ nhất mà tôi từng có. Chất lượng rất kém và sản phẩm hoàn toàn không giống như mô tả.",
    "Sau một tuần sử dụng, tôi nhận thấy nhiều vấn đề. Pin hết rất nhanh và thiết bị liên tục bị nóng, khiến nó không thể sử dụng được.",
    "Tôi hối hận vì đã mua món này. Hàng giao trễ, bao bì bị hỏng và bản thân sản phẩm thì đã bị hỏng sẵn.",
    "Mặc dù có nhiều đánh giá tích cực, trải nghiệm của tôi lại rất tệ. Sản phẩm bị lỗi và tôi đã lãng phí tiền vào một thứ không hoạt động.",

    # Nhóm 2: Dự đoán lớp 1 (tích cực)
    "Tôi hoàn toàn hài lòng với lần mua này. Sản phẩm hoạt động hoàn hảo và chất lượng vượt xa mong đợi của tôi ở mọi khía cạnh.",
    "Đây là một trong những sản phẩm tốt nhất mà tôi từng mua online. Hàng giao nhanh, đóng gói cẩn thận và hoạt động đúng như quảng cáo.",
    "Tôi đã sử dụng sản phẩm này được một tháng và rất hài lòng. Thời lượng pin ấn tượng và sản phẩm cực kỳ dễ sử dụng.",
    "Dịch vụ khách hàng rất tuyệt vời. Khi tôi gặp một vấn đề nhỏ, họ phản hồi ngay lập tức và gửi cho tôi sản phẩm thay thế mà không gặp bất kỳ rắc rối nào.",
    "Tôi rất khuyến khích sản phẩm này cho bất kỳ ai đang tìm kiếm chất lượng và độ tin cậy. Nó đã giúp công việc hàng ngày của tôi trở nên dễ dàng và thú vị hơn rất nhiều.",

    # Nhóm 3: Dự đoán chung chung (trung tính, không rõ ràng)
    "Sản phẩm được giao đúng hạn và đúng như mô tả.",
    "Tôi đã sử dụng sản phẩm được vài ngày. Nó hoạt động như mong đợi.",
    "Bao bì bình thường, không có gì đặc biệt.",
    "Tôi nhận được sản phẩm vào ngày hôm qua và chưa sử dụng nhiều.",
    "Sản phẩm ổn, không tốt cũng không xấu."
]


# 1. Tiền xử lý các câu test (giống hàm preprocess_text bạn đã dùng)
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z0-9À-ỹ\s]', '', text)
    return text



test_sentences_processed = [preprocess_text(s) for s in test_sentences]

# 2. Encode embedding cho các câu test
test_sentences_emb = embedder.encode(test_sentences_processed, show_progress_bar=False)

# 3. Dự đoán với SVM đã train
y_test_pred = grid.predict(test_sentences_emb)

# 4. Hiển thị kết quả
for text, pred in zip(test_sentences, y_test_pred):
    print(f"Review: {text} -> Dự đoán: {pred} sao")

Review: Tôi cực kỳ thất vọng với sản phẩm này. Nó ngừng hoạt động chỉ sau hai ngày và dịch vụ khách hàng không hỗ trợ gì khi tôi cố gắng yêu cầu đổi trả. -> Dự đoán: 0 sao
Review: Đây chắc chắn là lần mua hàng tệ nhất mà tôi từng có. Chất lượng rất kém và sản phẩm hoàn toàn không giống như mô tả. -> Dự đoán: 0 sao
Review: Sau một tuần sử dụng, tôi nhận thấy nhiều vấn đề. Pin hết rất nhanh và thiết bị liên tục bị nóng, khiến nó không thể sử dụng được. -> Dự đoán: 0 sao
Review: Tôi hối hận vì đã mua món này. Hàng giao trễ, bao bì bị hỏng và bản thân sản phẩm thì đã bị hỏng sẵn. -> Dự đoán: 0 sao
Review: Mặc dù có nhiều đánh giá tích cực, trải nghiệm của tôi lại rất tệ. Sản phẩm bị lỗi và tôi đã lãng phí tiền vào một thứ không hoạt động. -> Dự đoán: 1 sao
Review: Tôi hoàn toàn hài lòng với lần mua này. Sản phẩm hoạt động hoàn hảo và chất lượng vượt xa mong đợi của tôi ở mọi khía cạnh. -> Dự đoán: 1 sao
Review: Đây là một trong những sản phẩm tốt nhất mà tôi từng mua online. Hàng giao nhan